In [1]:
import os
import numpy as np
import webvtt
from datasets import Dataset, concatenate_datasets, load_from_disk
from scipy.io import wavfile
import torch
import numpy as np
from transformers import WhisperProcessor, WhisperFeatureExtractor, WhisperTokenizer, DataCollatorForSeq2Seq
from transformers import WhisperForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments,DataCollatorForSeq2Seq
from dataclasses import dataclass
from typing import Any, Dict, List, Union



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [2]:
def vtt_to_text(vtt_file):
    transcript = []
    for caption in webvtt.read(vtt_file):
        transcript.append((caption.start, caption.end, caption.text.strip()))
    return transcript



def create_dataset(audio_file, transcript_data, target_sample_rate=16000):
    try:
        #Load the audio file with scipy
        sample_rate, audio_data = wavfile.read(audio_file)
        print(f"Audio data shape: {audio_data.shape}, Sample rate: {sample_rate}")
    except Exception as e:
        raise RuntimeError(f"Failed to load audio file {audio_file}: {e}")

    #Preparing the dataset
    data = []
    
    for start, end, text in transcript_data:
        #Converting time from HH:MM:SS to seconds
        start_sec = sum(float(x) * 60 ** i for i, x in enumerate(reversed(start.split(":"))))
        end_sec = sum(float(x) * 60 ** i for i, x in enumerate(reversed(end.split(":"))))
        
        #Calculating start and end frames based on the target sample rate
        start_frame = max(0, min(int(start_sec * sample_rate), len(audio_data)))
        end_frame = max(0, min(int(end_sec * sample_rate), len(audio_data)))

        #Cutting the audio chunk for the corresponding transcript
        audio_chunk = audio_data[start_frame:end_frame]

        #Ensure the audio chunk is in the correct format
        audio_chunk_list = audio_chunk.tolist()
        data.append({"audio": audio_chunk_list, "text": text})

    #Check if transcript data is empty
    if not data:
        print(f"No valid transcript data for {audio_file}.")
        return None
    
    #Creating the dataset
    dataset = Dataset.from_dict({
        "audio": [d["audio"] for d in data],
        "text": [d["text"] for d in data]
    })
    
    print(f"Created dataset for {audio_file} with {len(data)} entries.")
    return dataset


def find_subtitle_file(audio_filename, subtitle_folder):
    base_audio_name = os.path.splitext(audio_filename)[0]  
    # Searching for the subtitle file that contains the audio filename in their name
    for subtitle_filename in os.listdir(subtitle_folder):
        if base_audio_name in subtitle_filename and subtitle_filename.endswith('.vtt'):
            return os.path.join(subtitle_folder, subtitle_filename)
    
    return None


def process_videos(audio_folder, subtitle_folder):
    datasets = []

    for audio_file in os.listdir(audio_folder):
        if audio_file.endswith('.wav'):
            input_file_path = os.path.join(audio_folder, audio_file)

            #Find the corresponding subtitle file
            subtitle_file_path = find_subtitle_file(audio_file, subtitle_folder)
            if subtitle_file_path:
                print(f"Found subtitle file: {subtitle_file_path}")

                #Convert subtitles to text transcript
                transcript_data = vtt_to_text(subtitle_file_path)
                if transcript_data:
                    #Create the dataset using the existing WAV file
                    dataset = create_dataset(input_file_path, transcript_data)
                    if dataset:  #Check if the dataset is created
                        datasets.append(dataset)
                    else:
                        print(f"Failed to create dataset for {input_file_path}.")
                else:
                    print(f"No transcript data found for {subtitle_file_path}. The file may be empty or incorrectly formatted.")
            else:
                print(f"Subtitle file not found for {audio_file}. Expected naming convention: {os.path.splitext(audio_file)[0]}*.vtt")

    #Concatenate all datasets
    if datasets:
        combined_dataset = concatenate_datasets(datasets)
        print(f"Combined dataset created with {len(datasets)} individual datasets.")
        return combined_dataset
    else:
        print("No datasets were created.")
        return None
    
    
    
    
audio_path = 'audio_files_copy/'
subtitle_path = 'manual_sub_copy/'
combined_dataset = process_videos(audio_path, subtitle_path)

Found subtitle file: manual_sub_copy/Lecture12TheRatioRootandAlternatingSeriesTests_manual.en-j3PyPqV-e1s.vtt
Audio data shape: (57920598,), Sample rate: 16000
Created dataset for audio_files_copy/Lecture12TheRatioRootandAlternatingSeriesTests.wav with 661 entries.
Found subtitle file: manual_sub_copy/Lecture9LimsupLiminfandtheBolzano-WeierstrassTheorem_manual.en-j3PyPqV-e1s.vtt
Audio data shape: (70756011,), Sample rate: 16000
Created dataset for audio_files_copy/Lecture9LimsupLiminfandtheBolzano-WeierstrassTheorem.wav with 987 entries.
Found subtitle file: manual_sub_copy/Lecture8TheSqueezeTheoremandOperationsInvolvingConvergentSequences_manual.en-j3PyPqV-e1s.vtt
Audio data shape: (71873877,), Sample rate: 16000
Created dataset for audio_files_copy/Lecture8TheSqueezeTheoremandOperationsInvolvingConvergentSequences.wav with 923 entries.
Found subtitle file: manual_sub_copy/Lecture10TheCompletenessoftheRealNumbersandBasicPropertiesofInfiniteSeries_manual.en-j3PyPqV-e1s.vtt
Audio data s

In [3]:
from transformers import WhisperProcessor

# Load the processor for Whisper
processor = WhisperProcessor.from_pretrained("openai/whisper-base")

# Function to preprocess each entry in the dataset
def preprocess(batch):
    # Convert audio (list of samples) to input features (Mel spectrograms)
    audio = np.array(batch['audio'], dtype=np.float32)
    batch["input_features"] = processor.feature_extractor(audio, sampling_rate=16000).input_features[0]
    
    # Tokenize the transcription text
    batch["labels"] = processor.tokenizer(batch["text"]).input_ids

    return batch

# Apply the preprocessing to the entire dataset
processed_dataset = combined_dataset.map(preprocess, remove_columns=["audio", "text"])

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/8930 [00:00<?, ? examples/s]

In [4]:
processed_dataset

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 8930
})

In [5]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's appended later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [6]:
data_collator_n = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)


In [7]:
dataset_dict = processed_dataset.train_test_split(test_size=0.2)
train_dataset = dataset_dict['train']
eval_dataset = dataset_dict['test']

In [15]:
print("Number of entries in the dataset:", len(processed_dataset))
print("Dataset shape:", processed_dataset.shape)
print("Features of the dataset:", processed_dataset.features)


print("Number of entries in the train dataset:", len(train_dataset))
print("Train dataset shape:", train_dataset.shape)
print("Features of the train dataset:", train_dataset.features)


print("Number of entries in the eval dataset:", len(eval_dataset))
print("Eval dataset shape:", eval_dataset.shape)
print("Features of the eval dataset:", eval_dataset.features)

Number of entries in the dataset: 8930
Dataset shape: (8930, 2)
Features of the dataset: {'input_features': Sequence(feature=Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None), length=-1, id=None), 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}
Number of entries in the train dataset: 7144
Train dataset shape: (7144, 2)
Features of the train dataset: {'input_features': Sequence(feature=Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None), length=-1, id=None), 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}
Number of entries in the eval dataset: 1786
Eval dataset shape: (1786, 2)
Features of the eval dataset: {'input_features': Sequence(feature=Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None), length=-1, id=None), 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}


In [9]:
from transformers import WhisperForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments
# For clearing MPS memory (macbook issues lol)
torch.mps.empty_cache()  
# Load the Whisper model
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-finetuned",
    per_device_train_batch_size=8,
    #To use cpu since gpu memory was exceeded - mac issues again
    no_cuda=True,  
    num_train_epochs=1,  
    save_steps=500,  
    evaluation_strategy="steps",
    logging_steps=50,
    predict_with_generate=False, 
    gradient_accumulation_steps=4,  
)

# Create the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator_n,
    tokenizer=processor.tokenizer,
)

# Start training
trainer.train()


Step,Training Loss,Validation Loss
50,3.356200,0.596659
100,0.480200,0.404066
150,0.361000,0.349547
200,0.324800,0.323213


TrainOutput(global_step=223, training_loss=1.0463731534812482, metrics={'train_runtime': 21542.549, 'train_samples_per_second': 0.332, 'train_steps_per_second': 0.01, 'total_flos': 4.6284093259776e+17, 'train_loss': 1.0463731534812482, 'epoch': 1.0})

In [10]:
# Save the final model and processor
model.save_pretrained("./whisper-finetuned/final-model")
processor.save_pretrained("./whisper-finetuned/final-model")  